<a href="https://colab.research.google.com/github/Oussamayousre/DLL/blob/main/VideoDataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [74]:
!git clone https://github.com/metal3d/keras-video-generators.git  



Cloning into 'keras-video-generators'...
remote: Enumerating objects: 608, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (47/47), done.
remote: Total 608 (delta 43), reused 76 (delta 35), pack-reused 514
Receiving objects: 100% (608/608), 7.76 MiB | 27.86 MiB/s, done.
Resolving deltas: 100% (275/275), done.


In [ ]:
!pip install keras-video-generators


  Created wheel for keras-video-generators: filename=keras_video_generators-1.0.14-py3-none-any.whl size=12882 sha256=e03ed941dc0307e5495f09d761347e95350d9810c123a11a01df2c36c8582c02
  Stored in directory: /root/.cache/pip/wheels/bc/22/94/eda4e8caf00c0ffd3030fecbf2e0334b8a7f038f4451e20b00
Successfully built keras-video-generators


In [101]:
import math   # for mathematical operations
import matplotlib.pyplot as plt    # for plotting the images
%matplotlib inline
import pandas as pd
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from skimage.transform import resize   # for resizing images
from sklearn.model_selection import train_test_split
from glob import glob
from tqdm import tqdm

import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.regularizers import l2
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.utils import Sequence
from tensorflow.keras import datasets ,layers,models

from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report

from tensorflow.keras.models import Sequential
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from scipy.signal import argrelextrema
from imutils.video import VideoStream
#from keras_video import VideoFrameGenerator
from imutils import face_utils
import numpy as np
import argparse
#import imutils
import time
import dlib
import cv2
import os
import glob


# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model

# calculate a face embedding for each face in the dataset using facenet
from numpy import load
from numpy import expand_dims
from numpy import asarray
from numpy import savez_compressed
from keras.models import load_model

In [ ]:
"""arr = os.listdir("/content/gdrive/MyDrive/DataBase")
#label_train  = np.array([])
#Frame_train = np.array([])
i = 0
for path_1 in arr :

  
  
  #print(arr)
  #verify if the path is a directory or just a hidden file
  full_path_1 = f"/content/gdrive/MyDrive/DataBase/{path_1}" 
  isDirectory = os.path.isdir(full_path_1)
  if isDirectory : 
    arr_1 = os.listdir(full_path_1)
    
    for path_2 in arr_1 : 
      full_path_2 = f"/content/gdrive/MyDrive/DataBase/{path_1}/{path_2}" 
      arr_2 = os.listdir(full_path_2)
      
      for path_3 in arr_2 :
        i += 1 
        label_train  = []
        Frame_train = [] 
        #).reshape(0,1).astype(np.int64)
        label = int(os.path.splitext(path_3)[0])
        full_path_2 = f"/content/gdrive/MyDrive/DataBase/{path_1}/{path_2}/{path_3}"
        
        vs = cv2.VideoCapture(full_path_2)
        while True :
          ret, image = vs.read()
          if ret == False : 
            break
          image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
          image = cv2.resize(image, (224, 224))
          #image = image.img_to_array(image)
          label_train.append(label)
          Frame_train.append(image)
        #train classes 
        np.save(f'/content/gdrive/MyDrive/variables/X_dataset{i}.npy', Frame_train) # save
        #train labels
        np.save(f'/content/gdrive/MyDrive/variables/y_dataset{i}.npy', label_train) # save
        vs.release()
        cv2.destroyAllWindows()"""

In [2]:
#next step is to collect all the videos in one folder and take the first character of their name which is the label 
import numpy as np
import keras
#path to video's folder
path = '/content/gdrive/MyDrive/variables/Fold2_part2'
#
train_ID = os.listdir(path)[1:]
class VideoFrameGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self,train_ID, batch_size=1, dim=(56,56), shuffle=True ):
        'Initialization'
        self.batch_size = batch_size
        self.train_ID = train_ID   
        self.shuffle = shuffle
        self.dim = dim
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.train_ID) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.train_ID[k] for k in indexes]
        #list_IDs_temp = [20]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        X =  np.array([X])[0]
        y =  np.array([y])
        y = y.reshape(y.shape[1],1)
        randomize = np.arange(y.shape[0])
        np.random.shuffle(randomize)
        X = X[randomize]
        y = y[randomize]
        return X , y 

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.train_ID))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):

        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        #X = np.empty((self.batch_size, *self.dim, self.n_channels))
        #y = np.empty((self.batch_size), dtype=int)
        arr_1 = os.listdir(f'/content/gdrive/MyDrive/variables/Fold2_part2/{list_IDs_temp[0]}')
        label_train  = []
        Frame_train = [] 
        # Generate data
        for path_3 in arr_1 : 
          #).reshape(0,1).astype(np.int64)
          label = int(os.path.splitext(path_3)[0])
          full_path_2 = f"/content/gdrive/MyDrive/variables/Fold2_part2/{list_IDs_temp[0]}/{path_3}"
          vs = cv2.VideoCapture(full_path_2)
          nb_frame = 0
          while True :
            ret, image = vs.read()
            if ret == False  or nb_frame == 1000  : 
              break
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, self.dim)
            #image = image.img_to_array(image)
            label_train.append(label)
            Frame_train.append(image)
            nb_frame += 1
          vs.release()
          cv2.destroyAllWindows()
        return Frame_train, label_train
     


In [3]:
path = '/content/gdrive/MyDrive/variables/Fold2_part2'
train_ID = os.listdir(path)[1:]

# Parameters
params = {'dim': (56,56),
          'batch_size': 1,
          'shuffle': True}
training_generator = VideoFrameGenerator(train_ID, **params)
validation_generator = VideoFrameGenerator([19], **params)
 

In [ ]:
"""model = tf.keras.models.Sequential([
              tf.keras.layers.Conv2D(32, (3,3) , activation = 'relu' ,padding='same', input_shape = (56,56,3), kernel_regularizer=l2(0.01) ),
              tf.keras.layers.MaxPool2D(pool_size=(2, 2) ),
              #
              tf.keras.layers.Conv2D(32 , (3,3) ,padding='same', activation = 'relu',kernel_regularizer=l2(0.01)),
              tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
              #
              tf.keras.layers.Conv2D(64 , (3,3) , activation = 'relu', padding='same', kernel_regularizer=l2(0.01)),
              #tf.keras.layers.Dropout(0.2),
              tf.keras.layers.MaxPool2D(pool_size=(2, 2), padding='same'),
              #
            
              #
              #  tf.keras.layers.Conv2D(256 , (3,3) , activation = 'relu'),
              #  tf.keras.layers.Dropout(0.2),
              # tf.keras.layers.BatchNormalization() ,
              # tf.keras.layers.MaxPool2D(2,2),
              #
              tf.keras.layers.Flatten(),
              ##
              tf.keras.layers.Dense(64 , activation = 'relu', kernel_regularizer=l2(0.01)),
              #tf.keras.layers.Dropout(0.2),
              tf.keras.layers.Dense(11,activation = 'softmax' ) ]
               
    )
model.summary()"""


In [ ]:
"""opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer = opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(training_generator, validation_data=validation_generator, epochs = 10 )
#change 56 to 224 first thing to do 
history"""

In [3]:
import numpy as np
import keras
path = '/content/gdrive/MyDrive/variables/Fold2_part2'
train_ID = os.listdir(path)[1:]
class VideoFrameGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self,train_ID, batch_size=1, dim=(50,50), shuffle=True ):
        'Initialization'
        self.batch_size = batch_size
        self.train_ID = train_ID   
        self.shuffle = shuffle
        self.dim = dim
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.train_ID) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.train_ID[k] for k in indexes]
        #list_IDs_temp = [20]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        randomize = np.arange(len(x))
        np.random.shuffle(randomize)
        X = X[randomize]
        y = y[randomize]
        return np.array([X]), np.array([y])

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.train_ID))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):

        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        #X = np.empty((self.batch_size, *self.dim, self.n_channels))
        #y = np.empty((self.batch_size), dtype=int)
        arr_1 = os.listdir(f'/content/gdrive/MyDrive/variables/Fold2_part2/{list_IDs_temp[0]}')
        # Generate data
        for path_3 in arr_1 : 
          label_train  = []
          Frame_train = [] 
          #).reshape(0,1).astype(np.int64)
          label = int(os.path.splitext(path_3)[0])
          full_path_2 = f"/content/gdrive/MyDrive/variables/Fold2_part2/{list_IDs_temp[0]}/{path_3}"

          vs = cv2.VideoCapture(full_path_2)
          while True :
            ret, image = vs.read()
            if ret == False : 
              break
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, self.dim)
            #image = image.img_to_array(image)
            label_train.append(label)
            Frame_train.append(image)
          vs.release()
          cv2.destroyAllWindows()
        return Frame_train,label_train
     


In [63]:
x ,y = training_generator.__getitem__(0)


In [102]:
!pip install mtcnn

##############################################################
---



---



In [103]:
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

0.1.0


In [104]:
! pip install gdown

In [105]:
import gdown
!gdown --id 1PZ_6Zsy1Vb0s0JmjEmVd8FS99zoMCiN1

Downloading...
From: https://drive.google.com/uc?id=1PZ_6Zsy1Vb0s0JmjEmVd8FS99zoMCiN1
To: /content/facenet_keras.h5
92.4MB [00:00, 150MB/s]


In [106]:
from PIL import Image
from numpy import asarray
from mtcnn.mtcnn import MTCNN

# extract a single face from a given photograph
def extract_face(pixels, required_size=(160, 160)): 
    detector = MTCNN() 
    # create the detector, using default weights   
    # detect faces in the image
    results = detector.detect_faces(pixels)
    if len(results) == 0 : 
      return np.array([]).reshape(0, 160, 160, 3)
    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']
    # bug fix
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array
 
# load the photo and extract the face

In [107]:

model_facenet = tf.keras.models.load_model('/content/facenet_keras.h5')
print('Loaded Model')

Loaded Model


In [108]:
print(model_facenet.inputs)
print(model_facenet.outputs)

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]


In [109]:
# get the face embedding for one face
def get_embedding(model_facenet , face_pixels):
    # scale pixel values
    face_pixels = face_pixels.astype('float32')
    # standardize pixel values across channels (global)
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    # transform face into one sample
    samples = expand_dims(face_pixels, axis=0)
    # make prediction to get embedding
    yhat = model_facenet.predict(samples)
    return yhat[0]

### test code 

In [ ]:
arr_1 = os.listdir('/content/gdrive/MyDrive/variables/Fold3_part1/25')
# Generate data
for path_3 in arr_1 : 
  label_train  = np.array([])
  Frame_train = np.array([]).reshape(0,128)
  #).reshape(0,1).astype(np.int64)
  label = int(os.path.splitext(path_3)[0])
  full_path_2 = f"/content/gdrive/MyDrive/variables/Fold3_part1/25/{path_3}"
  #print(label)
  vs = cv2.VideoCapture(full_path_2)
  nb_frame = 0
  
  while True :
    detector = MTCNN()
    ret, image = vs.read()
    if ret == False or nb_frame == 3 : 

      break
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #print(image.shape)
    image = extract_face(image,required_size=(160, 160))
    print(image.shape)
    if len(image) == 0  :                
        #face_features = np.array([]).reshape(0,128)
        nb_frame += 1
    else : 
        face_features = get_embedding(model_facenet , image)             
        face_features = np.array(face_features) 
        face_features = face_features.reshape(1,128)
        label_train = np.append(label_train , label)
        Frame_train = np.append(Frame_train , face_features ,axis=0)
        nb_frame += 1
    #image = image.img_to_array(image)
    #label_train = np.append(label_train , label)
    #Frame_train = np.append(Frame_train , face_features ,axis=0)
     
  vs.release()
  cv2.destroyAllWindows()

In [122]:
#next step is to collect all the videos in one folder and take the first character of their name which is the label 
#path to videos folder
path = '/content/gdrive/MyDrive/variables/Fold2_part2'
#list of subfolders ID, os.listdir(path)[0] is for the validation data
train_ID = os.listdir(path)[1:]
class VideoFrameGenerator(Sequence):
    'Generates data for Keras'
    def __init__(self,train_ID , model_facenet ,detector = MTCNN(), batch_size=1, dim=(56,56), shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.train_ID = train_ID   
        self.shuffle = shuffle
        self.model_facenet = model_facenet
        self.dim = dim
        self.on_epoch_end()
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.train_ID) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.train_ID[k] for k in indexes]
        #list_IDs_temp = [20]
        # Generate data
        X, y = self.__data_generation(list_IDs_temp)
        #X =  np.array([X])[0]
        #y =  np.array([y])
        #y = y.reshape(y.shape[1],1)
        print(X.shape)
        print(y.shape)
        return X , y 

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.train_ID))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):

        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        #X = np.empty((self.batch_size, *self.dim, self.n_channels))
        #y = np.empty((self.batch_size), dtype=int)
        arr_1 = os.listdir(f'/content/gdrive/MyDrive/variables/Fold2_part2/{list_IDs_temp[0]}')
        # Generate data
        label_train  = np.array([]).reshape(0,1)
        Frame_train = np.array([]).reshape(0,128)
        for path_3 in arr_1 :
            #labeling the data using the title of videos 
            label = int(os.path.splitext(path_3)[0])
            full_path_2 = f'/content/gdrive/MyDrive/variables/Fold2_part2/{list_IDs_temp[0]}/{path_3}'
            # Opens the Video file
            
            vs = cv2.VideoCapture(full_path_2)
            nb_frame = 1        
            while True :    
                ret, image = vs.read()
                if ret == False or nb_frame == 3 : 
                    break
                
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                #print(image.shape)
                image = extract_face(image,required_size=(160, 160))
                #image = cv2.resize(image, (160,160))
                if len(image) == 0 :                
                    #face_features = np.array([]).reshape(0,128)
                    nb_frame += 1 
                else : 
                    face_features = get_embedding(self.model_facenet , image)             
                    face_features = np.array(face_features) 
                    face_features = face_features.reshape(1,128)
                    label_train = np.append(label_train , label)
                    Frame_train = np.append(Frame_train , face_features ,axis=0)
                    nb_frame += 1 
                #image = image.img_to_array(image)
                #label_train = np.append(label_train , label)
                #Frame_train = np.append(Frame_train , face_features ,axis=0)
                
            vs.release()
            cv2.destroyAllWindows()
        return Frame_train, label_train.reshape(label_train.shape[0]).astype(int)
     


In [123]:
path = '/content/gdrive/MyDrive/variables/Fold2_part2'
train_ID = os.listdir(path)[1:]

# Parameters
params = {'model_facenet' : model_facenet , 
          'batch_size': 1,
          'shuffle': True}
training_generator = VideoFrameGenerator(train_ID, **params)
validation_generator = VideoFrameGenerator([os.listdir(path)[0]], **params)
 

In [ ]:
x ,y = training_generator.__getitem__(0)


In [ ]:
y.shape

In [ ]:

model = models.Sequential([
          layers.Dense(256,activation='relu'),
          layers.Dense(128,activation='relu'),
          layers.Dense(64,activation='relu'),
          layers.Dense(32,activation='relu'),
          layers.Dense(16,activation='relu'),
          layers.Dense(11,activation='softmax'),

])
opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt,
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'])
history = model.fit(training_generator , validation_data= validation_generator, epochs = 2 )
history